In [1]:
#!pip install simpletransformers
from simpletransformers.ner import NERModel
import re

C:\Users\pizna\PycharmProjects\Test task\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model deploying

In [2]:
# Initialize the NER model
model_directory = "model/"
model = NERModel(model_type='bert', model_name=model_directory, use_cuda=False)

## Functions for converting in readeble format

In [3]:
def highlight_mountains(input_sentence, model_predict):
    mountains = []
    current_mountain = []

    for word in model_predict[0]:
        if 'B-MOUNT' in word.values():
            # If we find a new mountain name, store the previous one
            if current_mountain:
                mountains.append(' '.join(current_mountain)) 
                current_mountain = []
            current_mountain.append(list(word.keys())[0])
        elif 'I-MOUNT' in word.values() and current_mountain:  # Check for the continuation of a mountain name
            current_mountain.append(list(word.keys())[0])

    # Add the last mountain name if it exists
    if current_mountain:
        mountains.append(' '.join(current_mountain))

    # Replace mountain names in the input sentence with highlighted versions
    for mountain in mountains:
        input_sentence = re.sub(r'(?<!\w)' + re.escape(mountain) + r'(?!\w)', f'**{mountain}** (Mountains)', input_sentence)

    return input_sentence

In [4]:
def process_prediction(input_sentence):
    prediction, model_output = model.predict([input_sentence])

    # Highlight mountain names
    highlighted_sentence = highlight_mountains(input_sentence, prediction)
    print(highlighted_sentence) 

## Some tests 

In [5]:
input_sentence = "While climbing the majestic peaks of the Himalayas, including Mount Everest and Kanchenjunga, the mountaineers marveled at the breathtaking views and the unique challenges presented by each summit."
process_prediction(input_sentence)

Running Prediction: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it]

While climbing the majestic peaks of the **Himalayas,** (Mountains) including **Mount Everest** (Mountains) and **Kanchenjunga,** (Mountains) the mountaineers marveled at the breathtaking views and the unique challenges presented by each summit.


In [7]:
input_sentence = "Despite the challenges posed by the unpredictable weather, the expedition team successfully reached the summit of K2, one of the most treacherous peaks in the world."
process_prediction(input_sentence)

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  7.73it/s]

Despite the challenges posed by the unpredictable weather, the expedition team successfully reached the summit of **K2,** (Mountains) one of the most treacherous peaks in the world.


In [8]:
input_sentence = "Mont Blanc dominates the Alps as its tallest peak."
process_prediction(input_sentence)

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  6.49it/s]

**Mont Blanc** (Mountains) dominates the **Alps** (Mountains) as its tallest peak.


In [9]:
input_sentence = "Hoverla is the highest point in Ukraine."
process_prediction(input_sentence)

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  8.20it/s]

**Hoverla** (Mountains) is the highest point in Ukraine.
